## Import packages

In [5]:
import utils
import tensorflow as tf

## Data augmentation parameters

To fix the imbalanced classed problem, you want the augmented data to have a number of samples for species $i$ equal to:
$$ \frac{N}{n_{i}} \times \frac{C}{K} $$
with:
- $N$ being the total size of the original dataset
- $n_i$ the number of samples for species $i$ in the original dataset
- $C$ the augmentation factor
- $K$ the number of species, i.e K = 8

In [3]:
BALANCE_CLASSES: bool = True # Should the augmented data have as many samples in each species ?
AUGMENTATION_FACTOR: float = 5. # The size factor that we want to multiply our data with
RNG_SEED: int = 42 # The seed for the random effects of data augmentation

# Augmented Data Generator parameters
# Transformations
ROTATION_RANGE: float = 30. # Max rotation in degrees
ZOOM_RANGE: float = 0.3 # Max zoom
WIDTH_SHIFT_RANGE: int = 20 # Horizontal shift (in pixels)
HEIGHT_SHIFT_RANGE: int = 20 # Vertical shift (in pixels)
HORIZONTAL_FLIP: bool = True # Horizontal flip
VERTICAL_FLIP: bool = True # Vertical flip
BRIGHTNESS_SHIFT_RANGE: list[float] = [0.5, 1.2] # Minimum and max brightness scaling
SHEAR_RANGE: float = 5. # Max shear in degrees

FILL_MODE: str = "reflect" # Filling method for out-of-border pixels

96